In [223]:
# import packages
from bs4 import BeautifulSoup
import requests
import csv
import numpy as np
from pandas import DataFrame

In [224]:
# retrieve data from website
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [225]:
schema = rows[0].find_all('th')
rows = soup.find_all('tr')

In [226]:
# get heading column names
col_names = []
for col in schema:
    col_names.append(col.text.replace('\n', ''))

In [227]:
# iterate through each row to retrieve data
postcode_list = []
borough_list = []
neighbourhood_list =[]

for index in range(len(rows)):
    if index>0:
        row = rows[index].find_all('td')
        if(len(row)==3):
            postcode_list.append(row[0].text)
            borough_list.append(row[1].text)
            neighbourhood_list.append(row[2].text.replace('\n', ''))

In [228]:
# create pandas dataframe from retrieved data
data = {col_names[0]: postcode_list,
        col_names[1]: borough_list,
        col_names[2]: neighbourhood_list,
        }

df = DataFrame(data, columns= col_names)
df[:5]

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [229]:
# filtered out rows where Borough column is not assigned
df_borough_fil = df[df['Borough']!='Not assigned']


In [230]:
# fill neighbouhood column with borough information when neighbourhood is not assigned
df_borough_fil['Neighbourhood'] = np.where(df_borough_fil['Neighbourhood'] == 'Not assigned', df_borough_fil['Borough'], df_borough_fil['Neighbourhood'])


C:\Users\wchan\Anaconda3\envs\envpy3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [231]:
df_borough_fil_grp_pc = df_borough_fil.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).reset_index()
df_borough_fil_grp_pc[:10]

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]
5,M1J,Scarborough,[Scarborough Village]
6,M1K,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]"
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]"
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]"
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]"


In [232]:
df_borough_fil_grp_pc.shape

(103, 3)